In [1]:
# from ggv.utils.context import create_default_context
# import ggv.utils.context
import res
import pandas as pd 
import numpy as np
import geopandas as gpd
#context = create_default_context()
from shapely.wkt import loads
from shapely.geometry import Point
import shapely
from ast import literal_eval
pd.options.display.max_colwidth=20

# Service instantiate

In [2]:
google_sheets=context.get_client("google_sheets")
# big_query=context.get_ds("ANALYTICS_BIG_QUERY")
# athena_db=context.get_ds("ANALYTICS_ATHENA_DB")

NameError: name 'context' is not defined

# Price Route Table From Hospital 

In [3]:

def get_price_tbl():
    big_query=context.get_ds("ANALYTICS_BIG_QUERY")
#     query_string=(
#     """
#     SELECT
#       from_zone_id,
#       from_z.name from_zone_name,
#       from_p.coordinates from_zone_coordinates,
#       to_zone_id,
#       to_z.name to_zone_name,
#       to_p.coordinates to_zone_coordinates,
#       price,
#       CASE
#         WHEN delivery_type_cd=0 THEN "Same Day"
#         WHEN delivery_type_cd=1 THEN "Speed"
#         WHEN delivery_type_cd=2 THEN "Instant"
#       END delivery_type,
#     FROM ((
#         WITH
#           price_tbl AS (
#           SELECT
#             from_zone_id,
#             to_zone_id,
#             price,
#             version,
#             delivery_type_cd,
#             ROW_NUMBER() OVER (PARTITION BY from_zone_id, to_zone_id, delivery_type_cd ORDER BY version DESC ) row_num
#           FROM
#             gogox-analytics-prod.delivery_db.zone_to_zone_prices)
#         SELECT
#           from_zone_id,
#           to_zone_id,
#           price,
#           version,
#           delivery_type_cd
#         FROM
#           price_tbl
#         WHERE
#           row_num=1 )) sub1
#     LEFT JOIN
#       `gogox-analytics-prod.delivery_db.zones` from_z
#     ON
#       sub1.from_zone_id=from_z.id
#     LEFT JOIN
#       `gogox-analytics-prod.delivery_db.zones` to_z
#     ON
#       sub1.to_zone_id=to_z.id
#     LEFT JOIN
#       `gogox-analytics-prod.delivery_db.polygons` from_p
#     ON
#       sub1.from_zone_id=from_p.zone_id
#     LEFT JOIN
#       `gogox-analytics-prod.delivery_db.polygons` to_p
#     ON
#       sub1.to_zone_id=to_p.zone_id
#     WHERE
#       ST_COVERS(from_p.coordinates,ST_GEOGPOINT(114.18407,22.26326)) IS TRUE-- and ST_COVERS(to_p.coordinates,ST_GEOGPOINT(114.23612,22.33642)) is TRUE
#     ORDER BY
#       from_zone_id,
#       to_zone_id,
#       delivery_type_cd
#     """ )
#     return big_query.df_from_sql(query_string)

price_tbl_df=pd.read_csv('price.csv')


In [ ]:
SELECT
#       from_zone_id,
#       from_z.name from_zone_name,
#       from_p.coordinates from_zone_coordinates,
#       to_zone_id,
#       to_z.name to_zone_name,
#       to_p.coordinates to_zone_coordinates,
#       price,
#       CASE
#         WHEN delivery_type_cd=0 THEN "Same Day"
#         WHEN delivery_type_cd=1 THEN "Speed"
#         WHEN delivery_type_cd=2 THEN "Instant"
#       END delivery_type,
#     FROM ((
#         WITH
#           price_tbl AS (
#           SELECT
#             from_zone_id,
#             to_zone_id,
#             price,
#             version,
#             delivery_type_cd,
#             ROW_NUMBER() OVER (PARTITION BY from_zone_id, to_zone_id, delivery_type_cd ORDER BY version DESC ) row_num
#           FROM
#             gogox-analytics-prod.delivery_db.zone_to_zone_prices)
#         SELECT
#           from_zone_id,
#           to_zone_id,
#           price,
#           version,
#           delivery_type_cd
#         FROM
#           price_tbl
#         WHERE
#           row_num=1 )) sub1
#     LEFT JOIN
#       `gogox-analytics-prod.delivery_db.zones` from_z
#     ON
#       sub1.from_zone_id=from_z.id
#     LEFT JOIN
#       `gogox-analytics-prod.delivery_db.zones` to_z
#     ON
#       sub1.to_zone_id=to_z.id
#     LEFT JOIN
#       `gogox-analytics-prod.delivery_db.polygons` from_p
#     ON
#       sub1.from_zone_id=from_p.zone_id
#     LEFT JOIN
#       `gogox-analytics-prod.delivery_db.polygons` to_p
#     ON
#       sub1.to_zone_id=to_p.zone_id
#     WHERE
#       ST_COVERS(from_p.coordinates,ST_GEOGPOINT(114.18407,22.26326)) IS TRUE-- and ST_COVERS(to_p.coordinates,ST_GEOGPOINT(114.23612,22.33642)) is TRUE
#     ORDER BY
#       from_zone_id,
#       to_zone_id,
#       delivery_type_cd

In [4]:
# price_tbl_df=get_price_tbl()
price_tbl_df.drop(columns=['Unnamed: 0'],inplace=True)

In [5]:
#convert normal dataframe to geo dataframe
price_tbl_gdf=gpd.GeoDataFrame(price_tbl_df,geometry=price_tbl_df['to_zone_coordinates'].apply(loads))

In [6]:
price_tbl_gdf.head()

,from_zone_id,from_zone_name,from_zone_coordinates,to_zone_id,to_zone_name,to_zone_coordinates,price,delivery_type,geometry
0,162,司徒拔道,POLYGON((114.175...,1,三門仔,POLYGON((114.197...,0.0,Same Day,POLYGON ((114.19...
1,162,司徒拔道,POLYGON((114.175...,1,三門仔,POLYGON((114.197...,0.0,Speed,POLYGON ((114.19...
2,162,司徒拔道,POLYGON((114.175...,1,三門仔,POLYGON((114.197...,145.0,Instant,POLYGON ((114.19...
3,162,司徒拔道,POLYGON((114.175...,2,上水,POLYGON((114.123...,85.0,Same Day,POLYGON ((114.12...
4,162,司徒拔道,POLYGON((114.175...,2,上水,POLYGON((114.123...,105.0,Speed,POLYGON ((114.12...


In [7]:
price_tbl_gdf[price_tbl_gdf['delivery_type']=="Same Day"][['price','to_zone_name','delivery_type','geometry']].set_index('to_zone_name')

,price,delivery_type,geometry
to_zone_name,,,
三門仔,0.0,Same Day,POLYGON ((114.19...
上水,85.0,Same Day,POLYGON ((114.12...
上環,40.0,Same Day,POLYGON ((114.14...
中文大學,80.0,Same Day,POLYGON ((114.19...
中環,40.0,Same Day,POLYGON ((114.15...
...,...,...,...
科學園,90.0,Same Day,POLYGON ((114.20...
屯門內河碼頭,0.0,Same Day,POLYGON ((113.94...
大棠郊野公園,0.0,Same Day,POLYGON ((114.02...


# Getting the Clients Address List

In [8]:
#red_cross_gift_df=google_sheets.get_range_as_df("1aE7iwJQUHIXzVS-ii4GGmX-c7YLX0KD-YY8KvA8xEkU",sheet_range="Sheet1!A1:I61",header_row=0)

In [227]:
def client_address_list(sheetID,sheetRange,header_row=0):
    google_sheets=context.get_client("google_sheets")
    return google_sheets.get_range_as_df(sheetID,sheetRange,header_row)

In [228]:
# ghk_order_df=client_address_list('1AFApd_uuzObqjo3Ed552OgrnZYQnPLlg0KTP16OYexc',"16/9/2022")
ghk_order_df=pd.read_csv('GHK.csv')
#ghk_order_df.drop(columns=['Unnamed: 0'],inplace=True)

In [229]:
ghk_order_df.head()

,Delivery Date,Case No.,Patient Name,Patient Phone Number,Address,Formatted Address - Chinese,Region - Chinese,18-District - Chinese,Latitude,Longitude,Price(Same Day),Price(Speed),Price(Instant),Destination Zone
0,16/09/2022,220916001,CHIU Kin Shing,91788169,Flat 2104 YUET T...,新界元朗區天水圍天華路39號天悅...,新界,元朗區,22.46282,113.99919,80.0,95.0,155.0,天水圍
1,16/09/2022,220916002,WONG Siu Yuk,68283388,新界 清水灣甘澍路115號 CA...,新界西貢區清水灣甘澍路6號CAS...,新界,西貢區,22.33372,114.24093,0.0,0.0,125.0,井欄樹
2,16/09/2022,220916003,CHAN Wai Yung,64038803,Floor 8 Flat 803...,九龍觀塘區碧雲道223號德田邨德...,九龍,觀塘區,22.31019,114.23837,55.0,70.0,100.0,藍田
3,16/09/2022,220916004,WAN Suk Han,63068976,Floor 46 Flat A ...,新界西貢區將軍澳景嶺路8號都會駅3座,新界,西貢區,22.30432,114.25217,80.0,95.0,110.0,將軍澳
4,16/09/2022,220916005,LO Chi Kong,56909565,九龍 豐盛街32號 瓊山苑 A座...,九龍黃大仙區豐盛街32號瓊山苑瓊...,九龍,黃大仙區,22.34209,114.21060,55.0,70.0,100.0,鑽石山


In [230]:
ghk_order_df=ghk_order_df.loc[3037:,:]

In [231]:
ghk_order_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 3009 to 3036
Data columns (total 14 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Delivery Date                28 non-null     object 
 1   Case No.                     28 non-null     int64  
 2   Patient Name                 28 non-null     object 
 3   Patient Phone Number         27 non-null     object 
 4   Address                      28 non-null     object 
 5   Formatted Address - Chinese  0 non-null      object 
 6   Region - Chinese             0 non-null      object 
 7   18-District - Chinese        0 non-null      object 
 8   Latitude                     0 non-null      float64
 9   Longitude                    0 non-null      float64
 10  Price(Same Day)              0 non-null      float64
 11  Price(Speed)                 0 non-null      float64
 12  Price(Instant)               0 non-null      float64
 13  Destination Zone 

In [232]:
#ghk_order_df.rename(columns={'':'Pick Up Time'},inplace=True)

# ghk_order_df.drop(columns=['Price(Same Day)',
#        'Price(Speed)', 'Price(Instant)', 'Destination Zone'],inplace=True)

In [233]:
ghk_order_df.head()

,Delivery Date,Case No.,Patient Name,Patient Phone Number,Address,Formatted Address - Chinese,Region - Chinese,18-District - Chinese,Latitude,Longitude,Price(Same Day),Price(Speed),Price(Instant),Destination Zone
3009,31/10/2022,221031025,CHENG Suk Ling,95597431,新界 大埔寶湖道3號 寶湖花園 ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3010,31/10/2022,221031026,TSUI Wai Leung S...,51136622,Floor 9 Flat D P...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3011,31/10/2022,221031027,LO Lai Ching,852-94465232,Floor 28 Flat 19...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3012,31/10/2022,221031028,WAN Wai Hung,852-93057066,嘉才里54號1/F樓,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3013,31/10/2022,221031029,WONG Oi Wa,96898693,Floor 23 Flat E ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Gov API Querying Query


In [234]:
import requests
import urllib3
import xmltodict
import json


In [235]:
def address_query(df,column):
    param={"q":df[column],"Accept":"application/json","n":"1"}#,"3d":"1"} 
    # return the json of the top result with floor and unit detail
    q_url=requests.get('https://www.als.ogcio.gov.hk/lookup?',params=param)
    result=xmltodict.parse(q_url.text)
    df["Query XML"]=result
    
    return df

In [213]:
# for i,v in enumerate(xml_ser):
#     if 'AddressLookupResult' in v:
#         print(True)
#         print (i,v,"\n")



In [236]:
ghk_order_df=ghk_order_df.apply(lambda x:address_query(x,"Address"),axis=1)
ghk_order_df.head()



,Delivery Date,Case No.,Patient Name,Patient Phone Number,Address,Formatted Address - Chinese,Region - Chinese,18-District - Chinese,Latitude,Longitude,Price(Same Day),Price(Speed),Price(Instant),Destination Zone,Query XML
3009,31/10/2022,221031025,CHENG Suk Ling,95597431,新界 大埔寶湖道3號 寶湖花園 ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'AddressLookupR...
3010,31/10/2022,221031026,TSUI Wai Leung S...,51136622,Floor 9 Flat D P...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'AddressLookupR...
3011,31/10/2022,221031027,LO Lai Ching,852-94465232,Floor 28 Flat 19...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'AddressLookupR...
3012,31/10/2022,221031028,WAN Wai Hung,852-93057066,嘉才里54號1/F樓,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'AddressLookupR...
3013,31/10/2022,221031029,WONG Oi Wa,96898693,Floor 23 Flat E ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'AddressLookupR...


In [51]:
#Region Extract From Gov API
def region_extract(query_col):
#         parsed_dict=eval(query_col)
        region=query_col['AddressLookupResult']['SuggestedAddress']['Address']['PremisesAddress']['ChiPremisesAddress']['Region']
        return region

In [23]:
#District Extract From GOV API
def district_extract(query_col):
#     parsed_dict=eval(query_col)
    district=query_col['AddressLookupResult']['SuggestedAddress']['Address']['PremisesAddress']['ChiPremisesAddress']['ChiDistrict']['DcDistrict']
    return district

In [24]:
#LatLong Extract From GOV API
def latlong_extract(df,query_col):
#     parsed_dict=eval(df[query_col])
    geo_detail=df[query_col]['AddressLookupResult']['SuggestedAddress']['Address']['PremisesAddress']['GeospatialInformation']
    df['Latitude']=geo_detail['Latitude']
    df['Longitude']=geo_detail['Longitude']
    return df


In [25]:
#Full Address Extract FOM GOV API
def full_address(query_col):
    detail_dict=query_col['AddressLookupResult']['SuggestedAddress']['Address']['PremisesAddress']['ChiPremisesAddress']
    block_no=block_quantifier=building_name=phase=estate_name=district=region=str()
    location_name=village_name=village_no_from=village_no_to=street_name=street_no_from=street_no_to=str()
    
    #block section   
    if 'ChiBlock' in detail_dict:
        if 'BlockNo' in detail_dict['ChiBlock']:
            block_no= detail_dict['ChiBlock']['BlockNo']
                
        if 'BlockDescriptor' in detail_dict['ChiBlock']:
            block_quantifier=detail_dict['ChiBlock']['BlockDescriptor']
    #building section        
    if 'BuildingName' in detail_dict:
        building_name=detail_dict['BuildingName']
        
    #Estate section   
    if 'ChiEstate' in detail_dict:
        if 'ChiPhase' in detail_dict['ChiEstate']:
            phase=detail_dict['ChiEstate']['ChiPhase']['PhaseName']
        if 'EstateName' in detail_dict['ChiEstate']:
            estate_name=detail_dict['ChiEstate']['EstateName']
    #District section
    if 'ChiDistrict' in detail_dict:
        district=detail_dict['ChiDistrict']['DcDistrict']
    if 'Region' in detail_dict:
        region=detail_dict['Region']

    addresslist=[region,district,estate_name,phase,building_name,block_no,block_quantifier] 
        
    if 'ChiVillage' in detail_dict:
        if 'LocationName' in detail_dict['ChiVillage']:
            location_name=detail_dict['ChiVillage']['LocationName']
        if 'VillageName' in detail_dict['ChiVillage']:
            village_name=detail_dict['ChiVillage']['VillageName']
        if 'BuildingNoFrom' in detail_dict['ChiVillage']:
            village_no_from=detail_dict['ChiVillage']['BuildingNoFrom']
        if 'BuildingNoTo' in detail_dict['ChiVillage']:
            village_no_to=detail_dict['ChiVillage']['BuildingNoTo']
        if villa_no_to!="":
            village_no_str=f"{villa_no_from}號-{villa_no_to}號"
        else:
            village_no_str=f"{villa_no_from}號" 
        addresslist=[region,district,location_name,village_name,village_no_str,estate_name,phase,building_name,block_no,block_quantifier]  
        
        
    if 'ChiStreet' in detail_dict:
        if 'LocationName' in detail_dict['ChiStreet']:
            location_name=detail_dict['ChiStreet']['LocationName']
        if 'StreetName' in detail_dict['ChiStreet']:
            street_name=detail_dict['ChiStreet']['StreetName']
        if 'BuildingNoFrom' in detail_dict['ChiStreet']:
            street_no_from=detail_dict['ChiStreet']['BuildingNoFrom']   
        if 'BuildingNoTo' in detail_dict['ChiStreet']:
            street_no_to=detail_dict['ChiStreet']['BuildingNoTo']
        if street_no_to!="":
            street_str=f"{street_no_from}號-{street_no_to}號"
        else:
            street_str=f"{street_no_from}號"  
        addresslist=[region,district,location_name,street_name,street_str,estate_name,phase,building_name,block_no,block_quantifier]  

        # df['Formatted Address - Chinese']=addresslist
        # print("".join(addresslist)) 
    return "".join(addresslist)
    #print(i,block_no,block_quantifer,building_name,phrase,estate_name,district,region)
    #print(i,region,district,estate_name,phrase,building_name,block_quantifer,block_no)
    #print (i,"{}-{}".format(street_no_from,street_no_to))
#     print (i,"{}{}{}{}\n{}\n{}{}\n{}{}{}".format(region,district,location_name,street_name,street_str,estate_name,phrase,building_name,block_no,block_quantifer))


In [26]:
#Floor Extract From GOV API
def floor_extract(df,query_col):
    add_detail=df[query_col]['AddressLookupResult']['SuggestedAddress']['Address']['PremisesAddress'] \
    ['ChiPremisesAddress'] 
    if 'Chi3dAddress' in add_detail:
        unit_detail=add_detail['Chi3dAddress']
        if isinstance(unit_detail,list):
            floor=unit_detail[0]['ChiFloor']
        else:
            floor=unit_detail['ChiFloor']
    else:
        floor=str()
    return floor


In [237]:
ghk_order_df['Region - Chinese']=ghk_order_df['Query XML'].apply(region_extract)
ghk_order_df['18-District - Chinese']=ghk_order_df['Query XML'].apply(district_extract)
ghk_order_df['Formatted Address - Chinese']=ghk_order_df['Query XML'].apply(full_address)
ghk_order_df=ghk_order_df.apply(lambda x:latlong_extract(x,"Query XML"),axis=1)
ghk_order_df=ghk_order_df.astype({"Latitude":"float64","Longitude":"float64"})

In [238]:
ghk_order_df=ghk_order_df.loc[0:,['Delivery Date','Case No.','Patient Name','Patient Phone Number','Address','Formatted Address - Chinese','Region - Chinese','18-District - Chinese','Latitude','Longitude','Query XML']]

In [239]:
#Convert to geodataframe
ghk_order_gdf=gpd.GeoDataFrame(ghk_order_df,geometry=[Point(x,y) for x,y in zip(ghk_order_df.Longitude,ghk_order_df.Latitude)])

In [240]:

#ghk_order_gdf.drop(columns=['Price(Same Day)','Price(Speed)','Price(Instant)','Destination Zone'],inplace=True)
ghk_order_gdf.head()

,Delivery Date,Case No.,Patient Name,Patient Phone Number,Address,Formatted Address - Chinese,Region - Chinese,18-District - Chinese,Latitude,Longitude,Query XML,geometry
3009,31/10/2022,221031025,CHENG Suk Ling,95597431,新界 大埔寶湖道3號 寶湖花園 ...,新界大埔區寶湖道3號寶湖花園寶湖道街市,新界,大埔區,22.44930,114.16853,{'AddressLookupR...,POINT (114.16853...
3010,31/10/2022,221031026,TSUI Wai Leung S...,51136622,Floor 9 Flat D P...,香港灣仔區英皇道6號-8號百利大廈,香港,灣仔區,22.28256,114.19246,{'AddressLookupR...,POINT (114.19246...
3011,31/10/2022,221031027,LO Lai Ching,852-94465232,Floor 28 Flat 19...,新界元朗區天水圍天瑞路73號天澤...,新界,元朗區,22.46838,113.99786,{'AddressLookupR...,POINT (113.99786...
3012,31/10/2022,221031028,WAN Wai Hung,852-93057066,嘉才里54號1/F樓,新界屯門區屯門嘉才里54號,新界,屯門區,22.37008,113.99689,{'AddressLookupR...,POINT (113.99689...
3013,31/10/2022,221031029,WONG Oi Wa,96898693,Floor 23 Flat E ...,九龍黃大仙區彩虹道242號采頤花園8座,九龍,黃大仙區,22.33560,114.20204,{'AddressLookupR...,POINT (114.20204...


# Merge Route Price and Clients List

In [219]:
# ghk_order_withroute_gdf=gpd.sjoin(ghk_order_gdf,price_tbl_gdf[price_tbl_gdf['delivery_type']=="Same Day"][['price','to_zone_name','delivery_type','geometry']]
#                       .rename(columns={'price':'Price (Same Day)'}),how='left',op='within')
# ghk_order_withroute_gdf.drop(columns="index_right").head(3)
# ghk_order_withroute_gdf.head()

In [241]:
cnt=1
for d_type in price_tbl_gdf['delivery_type'].unique():
    if cnt==1:
        ghk_order_withroute_gdf=gpd.sjoin(ghk_order_gdf,price_tbl_gdf[price_tbl_gdf['delivery_type']==d_type][["price","to_zone_name","geometry"]],how='left',op='within')
        ghk_order_withroute_gdf.drop(columns="index_right",inplace=True)
        ghk_order_withroute_gdf.rename(columns={'price':f"Price({d_type})"},inplace=True)
    else:
        ghk_order_withroute_gdf=gpd.sjoin(ghk_order_withroute_gdf,price_tbl_gdf[price_tbl_gdf['delivery_type']==d_type][["price","geometry"]],how='left',op='within')
        ghk_order_withroute_gdf.drop(columns='index_right',inplace=True)
        ghk_order_withroute_gdf.rename(columns={'price':f"Price({d_type})"},inplace=True)
    cnt+=1
                                                                                                    
ghk_order_withroute_gdf.head(10)




/Users/samho/Desktop/Anaconda/anaconda3/envs/geo_env/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3318: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/Users/samho/Desktop/Anaconda/anaconda3/envs/geo_env/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3318: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/Users/samho/Desktop/Anaconda/anaconda3/envs/geo_env/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3318: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


,Delivery Date,Case No.,Patient Name,Patient Phone Number,Address,Formatted Address - Chinese,Region - Chinese,18-District - Chinese,Latitude,Longitude,Query XML,geometry,Price(Same Day),to_zone_name,Price(Speed),Price(Instant)
3009,31/10/2022,221031025,CHENG Suk Ling,95597431,新界 大埔寶湖道3號 寶湖花園 ...,新界大埔區寶湖道3號寶湖花園寶湖道街市,新界,大埔區,22.44930,114.16853,{'AddressLookupR...,POINT (114.16853...,85.0,大埔,105.0,130.0
3010,31/10/2022,221031026,TSUI Wai Leung S...,51136622,Floor 9 Flat D P...,香港灣仔區英皇道6號-8號百利大廈,香港,灣仔區,22.28256,114.19246,{'AddressLookupR...,POINT (114.19246...,40.0,天后,55.0,65.0
3011,31/10/2022,221031027,LO Lai Ching,852-94465232,Floor 28 Flat 19...,新界元朗區天水圍天瑞路73號天澤...,新界,元朗區,22.46838,113.99786,{'AddressLookupR...,POINT (113.99786...,80.0,天水圍,95.0,155.0
3012,31/10/2022,221031028,WAN Wai Hung,852-93057066,嘉才里54號1/F樓,新界屯門區屯門嘉才里54號,新界,屯門區,22.37008,113.99689,{'AddressLookupR...,POINT (113.99689...,0.0,小欖,0.0,135.0
3013,31/10/2022,221031029,WONG Oi Wa,96898693,Floor 23 Flat E ...,九龍黃大仙區彩虹道242號采頤花園8座,九龍,黃大仙區,22.33560,114.20204,{'AddressLookupR...,POINT (114.20204...,55.0,新蒲崗,70.0,100.0
3014,31/10/2022,221031030,CHOI Fai Yeuk,92832108,Floor 14 Flat 2 ...,新界沙田區沙田車公廟路69號顯徑...,新界,沙田區,22.36090,114.17193,{'AddressLookupR...,POINT (114.17193...,80.0,大圍,95.0,120.0
3015,31/10/2022,221031031,LUO Binghui,852-95856622,清河村清平樓28樓 2818室,新界北區上水清曉路5號清河邨清平樓A座,新界,北區,22.49362,114.12577,{'AddressLookupR...,POINT (114.12577...,85.0,上水,105.0,140.0
3016,31/10/2022,221031032,YAU Wai Ying,93093141,荔枝角道泓景臺第三座41樓 F室,九龍深水埗區荔枝角道863號泓景...,九龍,深水埗區,22.33449,114.14758,{'AddressLookupR...,POINT (114.14758...,55.0,荔枝角,70.0,100.0
3017,31/10/2022,221031033,CHONG Mei Kuen,97368966,Floor 13 Flat 20...,新界葵青區葵涌荔景山路282號華...,新界,葵青區,22.34274,114.13750,{'AddressLookupR...,POINT (114.13750...,55.0,荔景,70.0,100.0
3018,31/10/2022,221031034,LEUNG Kim Yuen,98727922,Floor 21 Flat 16...,九龍黃大仙區慈雲山道55號慈民邨民裕樓,九龍,黃大仙區,22.34815,114.20383,{'AddressLookupR...,POINT (114.20383...,55.0,慈雲山,70.0,100.0


In [242]:
ghk_order_result_df=ghk_order_withroute_gdf.rename(columns={'to_zone_name':'Destination Zone'}) \
[['Delivery Date','Case No.','Patient Name','Patient Phone Number','Address','Formatted Address - Chinese','Region - Chinese','18-District - Chinese','Latitude','Longitude','Price(Same Day)', \
  'Price(Speed)','Price(Instant)','Destination Zone']]

In [243]:
ghk_order_result_df.tail()

,Delivery Date,Case No.,Patient Name,Patient Phone Number,Address,Formatted Address - Chinese,Region - Chinese,18-District - Chinese,Latitude,Longitude,Price(Same Day),Price(Speed),Price(Instant),Destination Zone
3032,31/10/2022,221031048,SEE Yin Yu,852-60196908,Floor 28 Flat G ...,新界沙田區沙田駿景路1號駿景園10座,新界,沙田區,22.40171,114.20278,80.0,95.0,120.0,火炭
3033,31/10/2022,221031049,TSE Pik Lan,95896991,Floor 1 Flat 103...,九龍觀塘區慶田街7號藍田邨第7期...,九龍,觀塘區,22.31176,114.23673,55.0,70.0,100.0,藍田
3034,31/10/2022,221031050,LOK Ut Seong,90766739,白石角逸瓏灣二期低6座地下B室,新界大埔區蝦地下7號蝦地下盧氏宗祠,新界,大埔區,22.46528,114.20383,0.0,0.0,145.0,三門仔
3035,31/10/2022,221031051,LEE Shuk Kwan,61996165,Floor 23 Flat A ...,新界西貢區將軍澳欣景路8號新都城...,新界,西貢區,22.32312,114.25871,80.0,95.0,110.0,將軍澳
3036,31/10/2022,221031052,WAN Ngan Fung,94716320,Floor 18 Flat 18...,新界大埔區大埔大埔太和路12號太...,新界,大埔區,22.45194,114.16261,85.0,105.0,130.0,太和


In [244]:
ghk_order_result_df[ghk_order_result_df.isnull().any(axis=1)]

,Delivery Date,Case No.,Patient Name,Patient Phone Number,Address,Formatted Address - Chinese,Region - Chinese,18-District - Chinese,Latitude,Longitude,Price(Same Day),Price(Speed),Price(Instant),Destination Zone
3025,31/10/2022,221031041,YAU Yun Chau,NaN,Floor 11 Flat D ...,新界沙田區馬鞍山保泰街18號天宇...,新界,沙田區,22.41524,114.22401,80.0,95.0,125.0,馬鞍山


In [245]:
ghk_order_result_df.isnull().mean()

Delivery Date                  0.000000
Case No.                       0.000000
Patient Name                   0.000000
Patient Phone Number           0.035714
Address                        0.000000
Formatted Address - Chinese    0.000000
Region - Chinese               0.000000
18-District - Chinese          0.000000
Latitude                       0.000000
Longitude                      0.000000
Price(Same Day)                0.000000
Price(Speed)                   0.000000
Price(Instant)                 0.000000
Destination Zone               0.000000
dtype: float64

In [246]:
ghk_order_result_df.reset_index(drop=True,inplace=True)

In [247]:
ghk_order_result_df.to_csv('GHK_Order.csv')

In [248]:
def update_clients_list(sheetID,sheet_range,result_df):
    google_sheets=context.get_client("google_sheets")
    return google_sheets.update_range_from_df(sheetID,sheet_range,result_df)

In [77]:
update_clients_list('1AFApd_uuzObqjo3Ed552OgrnZYQnPLlg0KTP16OYexc','16/9/2022',ghk_order_result_df)

Exception: Invalid range_start provided. Make sure you include a '!'.

# Re-validate address information

In [152]:
ghk_order_withroute_gdf.reset_index(drop=True,inplace=True)

In [153]:
ghk_order_withroute_gdf.tail()

,Delivery Date,Case No.,Patient Name,Patient Phone Number,Address,Formatted Address - Chinese,Region - Chinese,18-District - Chinese,Latitude,Longitude,Query XML,geometry,Price(Same Day),to_zone_name,Price(Speed),Price(Instant)
40,10/6/2022,221006075,YU Wai Yee,92885414,Floor 13 Flat 13...,九龍黃大仙區竹園道55號天馬苑駿...,九龍,黃大仙區,22.34330,114.18644,{'AddressLookupR...,POINT (114.18644...,55.0,黃大仙,70.0,100.0
41,10/6/2022,221006076,WONG Po Chu,97541682,香港 泰民街11號 樂翠臺 2...,香港東區泰民街11號樂翠臺2座,香港,東區,22.26798,114.23418,{'AddressLookupR...,POINT (114.23418...,40.0,柴灣,55.0,80.0
42,10/6/2022,221006077,TANG Yuk King,91850238,山景邨 景貴樓23樓 16室,新界屯門區屯門鳴琴路1號山景邨景...,新界,屯門區,22.39701,113.96442,{'AddressLookupR...,POINT (113.96442...,80.0,屯門,95.0,140.0
43,10/6/2022,221006078,LEE Yim Fan,98891283,"HOUSE 25, 24 STR...",新界大埔區大埔康樂園第二十街24...,新界,大埔區,22.46458,114.15454,{'AddressLookupR...,POINT (114.15454...,95.0,康樂園,120.0,130.0
44,10/6/2022,221006079,LAW Woon Hing,91011805,Floor 15 Flat H ...,香港東區太古城道17號太古城高山...,香港,東區,22.28597,114.21942,{'AddressLookupR...,POINT (114.21942...,40.0,太古,55.0,70.0


In [154]:
cnt=0
building_cnt=0
street_n_cnt=0
for i in range(len(ghk_order_withroute_gdf)):
    #parsed_dict=eval(ghk_order_withroute_gdf.loc[i,'Query XML'])
    detail_dict=ghk_order_withroute_gdf.loc[i,"Query XML"]['AddressLookupResult']['SuggestedAddress']["Address"]["PremisesAddress"]["ChiPremisesAddress"]
    block_no=block_quantifier=building_name=phase=estate_name=district=region=str()
    location_name=village_name=village_no_from=village_no_to=street_name=street_no_from=street_no_to=str()
    if 'ChiBlock' in detail_dict:
        if 'BlockNo' in detail_dict['ChiBlock']:
            block_no= detail_dict['ChiBlock']['BlockNo']
            print (i,"have block no")
                
        if 'BlockDescriptor' in detail_dict['ChiBlock']:
            block_quantifier=detail_dict['ChiBlock']['BlockDescriptor']
            print (i,"have block description")
            
    if 'BuildingName' in detail_dict:
        building_name=detail_dict['BuildingName']
        print (i,"have building name")
        building_cnt+=1
    
    if 'ChiEstate' in detail_dict:
        if 'ChiPhase' in detail_dict['ChiEstate']:
            phase=detail_dict['ChiEstate']['ChiPhase']['PhaseName']
            print (i,"have phase name")
        if 'EstateName' in detail_dict['ChiEstate']:
            estate_name=detail_dict['ChiEstate']['EstateName']
            print (i,"have estate name")

        
        
    if 'ChiDistrict' in detail_dict:
        district=detail_dict['ChiDistrict']['DcDistrict']
        print (i,"have dis name")
    if 'Region' in detail_dict:
        region=detail_dict['Region']
        print (i,"have region name")
        
    addresslist=[region,district,estate_name,phase,building_name,block_no,block_quantifier]
    if 'ChiVillage' in detail_dict:
        if 'LocationName' in detail_dict['ChiVillage']:
            location_name=detail_dict['ChiVillage']['LocationName']
        if 'VillageName' in detail_dict['ChiVillage']:
            village_name=detail_dict['ChiVillage']['VillageName']
        if 'BuildingNoFrom' in detail_dict['ChiVillage']:
            village_no_from=detail_dict['ChiVillage']['BuildingNoFrom']
        if 'BuildingNoTo' in detail_dict['ChiVillage']:
            village_no_to=detail_dict['ChiVillage']['BuildingNoTo']
        if villa_no_to!="":
            village_no_str=f"{villa_no_from}號-{villa_no_to}號"
        else:
            village_no_str=f"{villa_no_from}號" 
        addresslist=[region,district,location_name,village_name,village_no_str,estate_name,phase,building_name,block_no,block_quantifier]  
        
        
    if 'ChiStreet' in detail_dict:
        if 'LocationName' in detail_dict['ChiStreet']:
            location_name=detail_dict['ChiStreet']['LocationName']
        if 'StreetName' in detail_dict['ChiStreet']:
            street_name=detail_dict['ChiStreet']['StreetName']
            street_n_cnt+=1
        if 'BuildingNoFrom' in detail_dict['ChiStreet']:
            street_no_from=detail_dict['ChiStreet']['BuildingNoFrom']   
        if 'BuildingNoTo' in detail_dict['ChiStreet']:
            street_no_to=detail_dict['ChiStreet']['BuildingNoTo']
        if street_no_to!="":
            street_str=f"{street_no_from}號-{street_no_to}號"
        else:
            street_str=f"{street_no_from}號"  
        print (i,"have street detail")
        addresslist=[region,district,location_name,street_name,street_str,estate_name,phase,building_name,block_no,block_quantifier]  
    print (i,"".join(addresslist))
#     print (i,"{}{}{}{}\n{}\n{}{}\n{}{}{}".format(region,district,location_name,street_name,street_str,estate_name,phrase,building_name,block_no,block_quantifer))
    cnt+=1
print ("Total :",cnt,"Building Count=",building_cnt,"Street Name Count=",street_n_cnt)

0 have building name
0 have estate name
0 have dis name
0 have region name
0 have street detail
0 新界沙田區沙田豐禾里6號禾輋邨景和樓
1 have building name
1 have dis name
1 have region name
1 have street detail
1 新界荃灣區青山公路荃灣段303號-313號荃灣電話機樓
2 have block no
2 have block description
2 have estate name
2 have dis name
2 have region name
2 have street detail
2 九龍油尖旺區海庭道8號富榮花園7座
3 have block no
3 have block description
3 have building name
3 have estate name
3 have dis name
3 have region name
3 have street detail
3 新界元朗區天水圍天榮路3號天頌苑頌棋閣O座
4 have block no
4 have block description
4 have estate name
4 have dis name
4 have region name
4 have street detail
4 新界元朗區元朗 屏山沙井路11號金碧花園7座
5 have building name
5 have estate name
5 have dis name
5 have region name
5 have street detail
5 九龍觀塘區安田街18號平田邨平真樓
6 have block no
6 have block description
6 have building name
6 have estate name
6 have dis name
6 have region name
6 have street detail
6 新界屯門區屯門良運街3號建生邨樂生樓3座
7 have block no
7 have block description
7 have phase name
7 h

In [210]:
ghk_order_withroute_exp.head(20)

,Delivery Date,Case No.,Patient Name,Patient Phone Number,Address,Formatted Address - Chinese,Region - Chinese,18-District - Chinese,Latitude,Longitude,Price(Same Day),Price(Speed),Price(Instant),Destination Zone,Query XML
0,29/9/2022,220929035,LEUNG Chung Tak ...,94355776,Floor 3 Flat B 8...,香港灣仔區山光道25號香港賽馬會...,香港,灣仔區,22.26546,114.18675,40.0,55.0,65.0,跑馬地,{'AddressLookupR...
1,29/9/2022,220929036,MO Chun Fun,97266970,Floor 16 Flat 3 ...,新界葵青區葵涌興芳路177號葵芳...,新界,葵青區,22.35987,114.12951,55.0,70.0,105.0,葵涌,{'AddressLookupR...
2,29/9/2022,220929037,HUNG Chan Ming,95487965,新界 上水清曉路5號 清河邨清顯...,新界北區上水清曉路5號清河邨清顯...,新界,北區,22.49304,114.12740,85.0,105.0,140.0,上水,{'AddressLookupR...
3,29/9/2022,220929038,LEE Lan Heung,95179759,Floor 10 Flat 10...,NaN,九龍,觀塘區,22.29899,114.23801,55.0,70.0,100.0,油塘,{'AddressLookupR...
4,29/9/2022,220929039,LEE Yin Wah,68099750,Floor 36 Flat 22...,新界西貢區將軍澳彩明街2號善明邨...,新界,西貢區,22.30371,114.25016,80.0,95.0,110.0,將軍澳,{'AddressLookupR...
5,29/9/2022,220929040,CHEUNG Shun Wai,852-68228392,新界 大埔泰亨邨泰亨1號 泰亨豪...,新界大埔區泰亨1號泰亨豪園二十一座,新界,大埔區,22.47120,114.15019,0.0,0.0,135.0,九龍坑,{'AddressLookupR...
6,29/9/2022,220929041,WONG Yeuk Ha,62785051,Floor 9 Flat C B...,九龍黃大仙區瓊東街8號嘉峰臺2座,九龍,黃大仙區,22.34060,114.21090,55.0,70.0,100.0,鑽石山,{'AddressLookupR...
7,29/9/2022,220929042,NG Kam Kwan,62292348,Floor 18 Flat 18...,NaN,新界,元朗區,22.45670,113.99741,80.0,95.0,155.0,天水圍,{'AddressLookupR...
8,29/9/2022,220929043,SO Ka Yee,92428297,Floor 10 Flat 8 ...,新界西貢區將軍澳翠琳路2號景明苑...,新界,西貢區,22.32289,114.24983,80.0,95.0,110.0,將軍澳,{'AddressLookupR...
9,29/9/2022,220929044,YIP Chau Fai,94301853,Floor 10 Flat 1 ...,香港灣仔區威非路道31號佳景大廈,香港,灣仔區,22.28587,114.19058,40.0,55.0,65.0,天后,{'AddressLookupR...


In [256]:
cnt=0
building_cnt=0
street_name_cnt=0
for i in range(len(ghk_order_withroute_exp)):
    address_str=ghk_order_withroute_exp.loc[i,"Address"]
    suggested_results=eval(ghk_order_withroute_exp.loc[i,'Query XML'])['AddressLookupResult']['SuggestedAddress']
    print (f"Address String {i}:",address_str,"\n")
    if re.search(r"[\u4e00-\u9fa5].*",address_str):
    #query_dict=ghk_order_withroute_exp.loc[i,'Query XML']
        if isinstance(suggested_results,list):
            for result in suggested_results:
                chi_dict=result['Address']['PremisesAddress']['ChiPremisesAddress']
                print (chi_dict)
                
                if 'BuildingName'in chi_dict:
                    print(re.search(rf"{chi_dict['BuildingName']}",address_str))
                # if 'BuildingNoTo' in chi_dict['ChiStreet']:
                #     print(re.search(rf"{chi_dict['ChiStreet']['StreetName']}.*{chi_dict['ChiStreet']['BuildingNoFrom']}.?-{chi_dict['ChiStreet']['BuildingNoTo']}.?",address_str))
                # else:
                #     print(re.search(rf"{chi_dict['ChiStreet']['StreetName']}.*{chi_dict['ChiStreet']['BuildingNoFrom']}.*",address_str))
        else:
            chi_dict=suggested_results['Address']['PremisesAddress']['ChiPremisesAddress']
            if 'BuildingNoTo' in chi_dict['ChiStreet']:
                print(re.search(rf"{chi_dict['ChiStreet']['StreetName']}.*{chi_dict['ChiStreet']['BuildingNoFrom']}.?-{chi_dict['ChiStreet']['BuildingNoTo']}.?",address_str))
            else:
                print(re.search(rf"{chi_dict['ChiStreet']['StreetName']}.*{chi_dict['ChiStreet']['BuildingNoFrom']}.?",address_str))
            print (chi_dict)
                    
                    
#['AddressLookupResult']['SuggestedAddress']['Address']['PremisesAddress']['ChiPremisesAddress']
                            
                
#Floor 3 Flat B 8 SHAN KWONG ROAD, HAPPY VALLEY, HK            
#{'EngPremisesAddress': 
#{'EngEstate': {'EstateName': 'THE HONG KONG JOCKEY CLUB HAPPY VALLEY CLUBHOUSE'}, 
#'EngStreet': {'StreetName': 'SHAN KWONG ROAD', 'BuildingNoFrom': '25'},
#'EngDistrict': {'DcDistrict': 'WAN CHAI DISTRICT'}, 'Region': 'HK'}

#新界 上水清曉路5號 清河邨清顯樓21樓 7室
#{'ChiPremisesAddress': {'Region': '新界', 
#'ChiDistrict': {'DcDistrict': '北區'}, 
#'ChiStreet': {'LocationName': '上水', 'StreetName': '清曉路', 'BuildingNoFrom': '5'}, 
#'ChiEstate': {'EstateName': '清河邨'}, 
#'ChiBlock': {'BlockDescriptor': '座', 'BlockNo': '3&3A'}, 
#'BuildingName': '清顯樓(3 及 3A 座)'}

#0 {'Address': {'PremisesAddress': {'EngPremisesAddress': {'EngEstate': {'EstateName': 'THE HONG KONG JOCKEY CLUB HAPPY VALLEY CLUBHOUSE'}, 'EngStreet': {'StreetName': 'SHAN KWONG ROAD', 'BuildingNoFrom': '25'}, 'EngDistrict': {'DcDistrict': 'WAN CHAI DISTRICT'}, 'Region': 'HK'}, 'ChiPremisesAddress': {'Region': '香港', 'ChiDistrict': {'DcDistrict': '灣仔區'}, 'ChiStreet': {'StreetName': '山光道', 'BuildingNoFrom': '25'}, 'ChiEstate': {'EstateName': '香港賽馬會跑馬地會所'}}, 'GeoAddress': '3728914052T20050430', 'GeospatialInformation': {'Northing': '814071', 'Easting': '837286', 'Latitude': '22.26546', 'Longitude': '114.18675'}}}, 'ValidationInformation': {'Score': '71.43'}}
#0 {'Address': {'PremisesAddress': {'EngPremisesAddress': {'BuildingName': 'THE HONG KONG JOCKEY CLUB HAPPY VALLEY CLUBHOUSE', 'EngStreet': {'StreetName': 'SHAN KWONG ROAD', 'BuildingNoFrom': '25'}, 'EngDistrict': {'DcDistrict': 'WAN CHAI DISTRICT'}, 'Region': 'HK'}, 'ChiPremisesAddress': {'Region': '香港', 'ChiDistrict': {'DcDistrict': '灣仔區'}, 'ChiStreet': {'StreetName': '山光道', 'BuildingNoFrom': '25'}, 'BuildingName': '香港賽馬會跑馬地會所'}, 'GeoAddress': '3728914052T20050430', 'GeospatialInformation': {'Northing': '814071', 'Easting': '837286', 'Latitude': '22.26546', 'Longitude': '114.18675'}}}, 'ValidationInformation': {'Score': '71.43'}}
#0 {'Address': {'PremisesAddress': {'EngPremisesAddress': {'EngStreet': {'StreetName': 'SHAN KWONG ROAD', 'BuildingNoFrom': '8'}, 'EngDistrict': {'DcDistrict': 'WAN CHAI DISTRICT'}, 'Region': 'HK'}, 'ChiPremisesAddress': {'Region': '香港', 'ChiDistrict': {'DcDistrict': '灣仔區'}, 'ChiStreet': {'StreetName': '山光道', 'BuildingNoFrom': '8'}}, 'GeoAddress': '3703314392T20050430', 'GeospatialInformation': {'Northing': '814388', 'Easting': '837030', 'Latitude': '22.26832', 'Longitude': '114.18427'}}}, 'ValidationInformation': {'Score': '69.27'}}
#         else:
        
#         detail_dict=['Address']['PremisesAddress']['ChiPremisesAddress']
#         block_no=block_quantifier=building_name=phase=estate_name=district=region=str()
#         location_name=village_name=village_no_from=village_no_to=street_name=street_no_from=street_no_to=str()
#         if 'ChiBlock' in detail_dict:
#             if 'BlockNo' in detail_dict['ChiBlock']:
#                 block_no= detail_dict['ChiBlock']['BlockNo']

#             if 'BlockDescriptor' in detail_dict['ChiBlock']:
#                 block_quantifier=detail_dict['ChiBlock']['BlockDescriptor']

#         if 'BuildingName' in detail_dict:
#             building_name=detail_dict['BuildingName']
#             print (i,True)
#             building_cnt+=1

#         if 'ChiEstate' in detail_dict:
#             if 'ChiPhase' in detail_dict['ChiEstate']:
#                 phase=detail_dict['ChiEstate']['ChiPhase']['PhaseName']
#             if 'EstateName' in detail_dict['ChiEstate']:
#                 estate_name=detail_dict['ChiEstate']['EstateName']




#         if 'ChiDistrict' in detail_dict:
#             district=detail_dict['ChiDistrict']['DcDistrict']
#         if 'Region' in detail_dict:
#             region=detail_dict['Region']

#         addresslist=[region,district,estate_name,phase,building_name,block_no,block_quantifier]
#         if 'ChiVillage' in detail_dict:
#             if 'LocationName' in detail_dict['ChiVillage']:
#                 location_name=detail_dict['ChiVillage']['LocationName']
#             if 'VillageName' in detail_dict['ChiVillage']:
#                 village_name=detail_dict['ChiVillage']['VillageName']
#             if 'BuildingNoFrom' in detail_dict['ChiVillage']:
#                 village_no_from=detail_dict['ChiVillage']['BuildingNoFrom']
#             if 'BuildingNoTo' in detail_dict['ChiVillage']:
#                 village_no_to=detail_dict['ChiVillage']['BuildingNoTo']
#             if villa_no_to!="":
#                 village_no_str=f"{villa_no_from}號-{villa_no_to}號"
#             else:
#                 village_no_str=f"{villa_no_from}號" 
#             addresslist=[region,district,location_name,village_name,village_no_str,estate_name,phase,building_name,block_no,block_quantifier]  


#         if 'ChiStreet' in detail_dict:
#             if 'LocationName' in detail_dict['ChiStreet']:
#                 location_name=detail_dict['ChiStreet']['LocationName']
#             if 'StreetName' in detail_dict['ChiStreet']:
#                 street_name=detail_dict['ChiStreet']['StreetName']
#                 street_cnt+=1
#             if 'BuildingNoFrom' in detail_dict['ChiStreet']:
#                 street_no_from=detail_dict['ChiStreet']['BuildingNoFrom']   
#             if 'BuildingNoTo' in detail_dict['ChiStreet']:
#                 street_no_to=detail_dict['ChiStreet']['BuildingNoTo']
#             if street_no_to!="":
#                 street_str=f"{street_no_from}號-{street_no_to}號"
#             else:
#                 street_str=f"{street_no_from}號"  
            
#             addresslist=[region,district,location_name,street_name,street_str,estate_name,phase,building_name,block_no,block_quantifier]  
#     print (i,"".join(addresslist))
# #     print (i,"{}{}{}{}\n{}\n{}{}\n{}{}{}".format(region,district,location_name,street_name,street_str,estate_name,phrase,building_name,block_no,block_quantifer))
#     cnt+=1
# print ("Total :",cnt,"Building Count=",building_cnt

Address String 0: Floor 3 Flat B 8 SHAN KWONG ROAD, HAPPY VALLEY, HK 

Address String 1: Floor 16 Flat 3 KWAI CHI HOUSE, KWAI FONG ESTATE, 177 HING FONG ROAD, KWAI CHUNG, NT 

Address String 2: 新界 上水清曉路5號 清河邨清顯樓21樓 7室 

{'Region': '新界', 'ChiDistrict': {'DcDistrict': '北區'}, 'ChiStreet': {'LocationName': '上水', 'StreetName': '清曉路', 'BuildingNoFrom': '5'}, 'ChiEstate': {'EstateName': '清河邨'}, 'ChiBlock': {'BlockDescriptor': '座', 'BlockNo': '3&3A'}, 'BuildingName': '清顯樓(3 及 3A 座)'}
None
{'Region': '新界', 'ChiDistrict': {'DcDistrict': '北區'}, 'ChiStreet': {'LocationName': '上水', 'StreetName': '清曉路', 'BuildingNoFrom': '5'}, 'ChiEstate': {'EstateName': '清河邨'}, 'BuildingName': '清澤樓'}
None
{'Region': '新界', 'ChiDistrict': {'DcDistrict': '北區'}, 'ChiStreet': {'LocationName': '上水', 'StreetName': '清曉路', 'BuildingNoFrom': '5'}, 'ChiEstate': {'EstateName': '清河邨'}, 'BuildingName': '清照樓'}
None
{'Region': '新界', 'ChiDistrict': {'DcDistrict': '北區'}, 'ChiStreet': {'LocationName': '上水', 'StreetName': '清曉路', 'Buil

# Experimenting 

In [221]:
cnt=0
building_cnt=0
street_n_cnt=0
for i in range(len(ghk_order_withroute_exp)):
    parsed_dict=eval(ghk_order_withroute_exp.loc[i,'Query XML'])
    if isinstance(parsed_dict['AddressLookupResult']['SuggestedAddress'],list):
        for result in parsed_dict['AddressLookupResult']['SuggestedAddress']:
             print (i,"\n\n",result)
#     block_no=block_quantifier=building_name=phase=estate_name=district=region=str()
#     location_name=village_name=village_no_from=village_no_to=street_name=street_no_from=street_no_to=str()
#     if 'ChiBlock' in detail_dict:
#         if 'BlockNo' in detail_dict['ChiBlock']:
#             block_no= detail_dict['ChiBlock']['BlockNo']
#             print (i,"have block no")
                
#         if 'BlockDescriptor' in detail_dict['ChiBlock']:
#             block_quantifier=detail_dict['ChiBlock']['BlockDescriptor']
#             print (i,"have block description")
            
#     if 'BuildingName' in detail_dict:
#         building_name=detail_dict['BuildingName']
#         print (i,"have building name")
#         building_cnt+=1
    
#     if 'ChiEstate' in detail_dict:
#         if 'ChiPhase' in detail_dict['ChiEstate']:
#             phase=detail_dict['ChiEstate']['ChiPhase']['PhaseName']
#             print (i,"have phase name")
#         if 'EstateName' in detail_dict['ChiEstate']:
#             estate_name=detail_dict['ChiEstate']['EstateName']
#             print (i,"have estate name")

        
        
#     if 'ChiDistrict' in detail_dict:
#         district=detail_dict['ChiDistrict']['DcDistrict']
#         print (i,"have dis name")
#     if 'Region' in detail_dict:
#         region=detail_dict['Region']
#         print (i,"have region name")
        
#     addresslist=[region,district,estate_name,phase,building_name,block_no,block_quantifier]
#     if 'ChiVillage' in detail_dict:
#         if 'LocationName' in detail_dict['ChiVillage']:
#             location_name=detail_dict['ChiVillage']['LocationName']
#         if 'VillageName' in detail_dict['ChiVillage']:
#             village_name=detail_dict['ChiVillage']['VillageName']
#         if 'BuildingNoFrom' in detail_dict['ChiVillage']:
#             village_no_from=detail_dict['ChiVillage']['BuildingNoFrom']
#         if 'BuildingNoTo' in detail_dict['ChiVillage']:
#             village_no_to=detail_dict['ChiVillage']['BuildingNoTo']
#         if villa_no_to!="":
#             village_no_str=f"{villa_no_from}號-{villa_no_to}號"
#         else:
#             village_no_str=f"{villa_no_from}號" 
#         addresslist=[region,district,location_name,village_name,village_no_str,estate_name,phase,building_name,block_no,block_quantifier]  
        
        
#     if 'ChiStreet' in detail_dict:
#         if 'LocationName' in detail_dict['ChiStreet']:
#             location_name=detail_dict['ChiStreet']['LocationName']
#         if 'StreetName' in detail_dict['ChiStreet']:
#             street_name=detail_dict['ChiStreet']['StreetName']
#             street_n_cnt+=1
#         if 'BuildingNoFrom' in detail_dict['ChiStreet']:
#             street_no_from=detail_dict['ChiStreet']['BuildingNoFrom']   
#         if 'BuildingNoTo' in detail_dict['ChiStreet']:
#             street_no_to=detail_dict['ChiStreet']['BuildingNoTo']
#         if street_no_to!="":
#             street_str=f"{street_no_from}號-{street_no_to}號"
#         else:
#             street_str=f"{street_no_from}號"  
#         print (i,"have street detail")
#         addresslist=[region,district,location_name,street_name,street_str,estate_name,phase,building_name,block_no,block_quantifier]  
#     print (i,"".join(addresslist))
# #     print (i,"{}{}{}{}\n{}\n{}{}\n{}{}{}".format(region,district,location_name,street_name,street_str,estate_name,phrase,building_name,block_no,block_quantifer))
#     cnt+=1
# print ("Total :",cnt,"Building Count=",building_cnt,"Street Name Count=",street_n_cnt)

0 

 {'Address': {'PremisesAddress': {'EngPremisesAddress': {'EngEstate': {'EstateName': 'THE HONG KONG JOCKEY CLUB HAPPY VALLEY CLUBHOUSE'}, 'EngStreet': {'StreetName': 'SHAN KWONG ROAD', 'BuildingNoFrom': '25'}, 'EngDistrict': {'DcDistrict': 'WAN CHAI DISTRICT'}, 'Region': 'HK'}, 'ChiPremisesAddress': {'Region': '香港', 'ChiDistrict': {'DcDistrict': '灣仔區'}, 'ChiStreet': {'StreetName': '山光道', 'BuildingNoFrom': '25'}, 'ChiEstate': {'EstateName': '香港賽馬會跑馬地會所'}}, 'GeoAddress': '3728914052T20050430', 'GeospatialInformation': {'Northing': '814071', 'Easting': '837286', 'Latitude': '22.26546', 'Longitude': '114.18675'}}}, 'ValidationInformation': {'Score': '71.43'}}
0 

 {'Address': {'PremisesAddress': {'EngPremisesAddress': {'BuildingName': 'THE HONG KONG JOCKEY CLUB HAPPY VALLEY CLUBHOUSE', 'EngStreet': {'StreetName': 'SHAN KWONG ROAD', 'BuildingNoFrom': '25'}, 'EngDistrict': {'DcDistrict': 'WAN CHAI DISTRICT'}, 'Region': 'HK'}, 'ChiPremisesAddress': {'Region': '香港', 'ChiDistrict': {'DcDist

In [102]:
ghk_order_df.loc[1546,"Query XML"]
try:
    
except KeyError :
    


{'AddressLookupResult': {'RequestAddress': {'AddressLine': 'Floor 12 Flat 12 CHUNG YEW BUILDING, 75 KOK CHEUNG STREET, KOWLOON ((lift: 12/F, arrive 13/F)'}}}

In [155]:
ghk_order_withroute_exp=pd.read_csv("test.csv",index_col=False)

In [156]:
ghk_order_withroute_exp.drop(columns=["Unnamed: 0"],inplace=True)

ghk_order_withroute_exp.head()

,Delivery Date,Case No.,Patient Name,Patient Phone Number,Address,Formatted Address - Chinese,Region - Chinese,18-District - Chinese,Latitude,Longitude,Price(Same Day),Price(Speed),Price(Instant),Destination Zone,Query XML
0,29/9/2022,220929035,LEUNG Chung Tak ...,94355776,Floor 3 Flat B 8...,香港灣仔區山光道25號香港賽馬會...,香港,灣仔區,22.26546,114.18675,40.0,55.0,65.0,跑馬地,{'AddressLookupR...
1,29/9/2022,220929036,MO Chun Fun,97266970,Floor 16 Flat 3 ...,新界葵青區葵涌興芳路177號葵芳...,新界,葵青區,22.35987,114.12951,55.0,70.0,105.0,葵涌,{'AddressLookupR...
2,29/9/2022,220929037,HUNG Chan Ming,95487965,新界 上水清曉路5號 清河邨清顯...,新界北區上水清曉路5號清河邨清顯...,新界,北區,22.49304,114.12740,85.0,105.0,140.0,上水,{'AddressLookupR...
3,29/9/2022,220929038,LEE Lan Heung,95179759,Floor 10 Flat 10...,NaN,九龍,觀塘區,22.29899,114.23801,55.0,70.0,100.0,油塘,{'AddressLookupR...
4,29/9/2022,220929039,LEE Yin Wah,68099750,Floor 36 Flat 22...,新界西貢區將軍澳彩明街2號善明邨...,新界,西貢區,22.30371,114.25016,80.0,95.0,110.0,將軍澳,{'AddressLookupR...


In [163]:
# for v in ghk_order_withroute_exp[ghk_order_withroute_exp['Formatted Address - Chinese'].isna()]['Query XML']:
#     print (v,"\n")

In [162]:
# for v in ghk_order_withroute_exp[ghk_order_withroute_exp['Formatted Address - Chinese']!=""]['Query XML']:
#     print (v,"\n")

In [222]:
ghk_order_df=pd.read_csv("test.csv")
ghk_order_df.drop(columns=["Unnamed: 0"],inplace=True)
process_df=ghk_order_df.loc[:,['Address','Query XML']]




In [223]:

address_ser=process_df.iloc[:,0]
xml_ser=process_df.iloc[:,1]

In [224]:
ghk_order_df.loc[13,"Address"]

'Floor 6 Flat G BLK 6, PARK ISLAND, 8 PAK LAI ROAD, MA WAN, NT'

In [225]:
for i,v in enumerate(address_ser):
    print(i,v)
    print(re.search(r"(上水).+(清河邨)",v))

    

    

0 Floor 3 Flat B 8 SHAN KWONG ROAD, HAPPY VALLEY, HK
None
1 Floor 16 Flat 3 KWAI CHI HOUSE, KWAI FONG ESTATE, 177 HING FONG ROAD, KWAI CHUNG, NT
None
2 新界 上水清曉路5號 清河邨清顯樓21樓 7室
<re.Match object; span=(3, 14), match='上水清曉路5號 清河邨'>
3 Floor 10 Flat 1012 , WING TONG HSE, YAU TONG ESTATE, KOWLOON
None
4 Floor 36 Flat 22 BLK 1, SHIN CHI HOUSE (BLK 1), SHIN MING ESTATE, 2 CHOI MING STREET, TSEUNG KWAN O, NT
None
5 新界 大埔泰亨邨泰亨1號 泰亨豪園  26座一樓1樓
None
6 Floor 9 Flat C BLK 2, KINGSFORD TERRACE, 8 KING TUNG STREET, KOWLOON
None
7 Floor 18 Flat 1808 SHUI KWOK HOUSE, TIN SHUI (1) ESTATE, NT
None
8 Floor 10 Flat 8 BLK C, YUK KING HOUSE, KING MING COURT, 2 TSUI LAM ROAD, TSEUNG KWAN O, NT
None
9 Floor 10 Flat 1 BELLE HOUSE, 31 WHITFIELD ROAD, HK
None
10 Flat 1606 WAH KOON HOUSE, WAH SUM ESTATE, 18 YAT MING ROAD, FANLING, NT
None
11 粉嶺雞嶺村49號1F
None
12 Floor 18 Flat 15 BLK 8, KING YUNG HOUSE (BLK 8), KING LAM ESTATE, 38 PO LAM ROAD NORTH, TSEUNG KWAN O, NT
None
13 Floor 6 Flat G BLK 6, PARK ISLAND, 8 PAK LA

In [ ]:
finding=re.findall('YUNG YUEN HSE',test_str)
print(finding)

In [ ]:
xml_ser[0]

In [246]:
for i,v in enumerate(xml_ser):
    if i==11:
        parsed_info=eval(v)
        if isinstance(parsed_info['AddressLookupResult']['SuggestedAddress'],list):
            for item,result in enumerate(parsed_info['AddressLookupResult']['SuggestedAddress']):
                print (i)
                print (f"{i}.{item}\n {result['AddressLookupResult']['ChiStreet']['StreetName']['BuildingNoFrom']}")
                print ('-'*10)
            print ('Total:',item+1,"\n\n\n")
        else:
            print (i,'\n',parsed_info['AddressLookupResult']['SuggestedAddress']['Address']['PremisesAddress']['ChiPremisesAddress'],"\n")
            
#['ChiStreet']['StreetName']['BuildingNoFrom']

11 
 {'Region': '新界', 'ChiDistrict': {'DcDistrict': '北區'}, 'ChiStreet': {'LocationName': '粉嶺', 'StreetName': '雞嶺', 'BuildingNoFrom': '49'}} 



In [ ]:

cor_mapping = []
uni_cor = price_tbl_df[['to_zone_coordinates', 'to_zone_name']].drop_duplicates()

for n in range(len(ghk_order_df)):
    for x in uni_cor.index:
        if shapely.wkt.loads(uni_cor['to_zone_coordinates']\
                             .loc[x]).contains(Point(float(ghk_order_df.iloc[n]['Longitude']),float( ghk_order_df.iloc[n]['Latitude']))):
            cor_mapping.append([ghk_order_df['Case No.'].iloc[n], uni_cor['to_zone_name'].loc[x]]
                              )

ghk_order_df = ghk_order_df.merge(pd.DataFrame(cor_mapping), how = 'left', left_on = 'Case No.', right_on = 0)
ghk_order_result_df=ghk_order_df.rename(columns={1:'Destination Zone'}) \
[['Delivery Date','Case No.','Patient Name','Patient Phone Number','Address','Formatted Address - Chinese','Region - Chinese','18-District - Chinese','Latitude','Longitude','Destination Zone']]
ghk_order_result_df.head(4)

In [ ]:
def merge(price_df,lat_column,lon_column):
    price_df['
    return df

In [ ]:
enpoly = 100
polygon = [[np.sin(x)+0.5,np.cos(x)+0.5] for x in np.linspace(0,2*np.pi,lenpoly)[:-1]]


In [ ]:
uni_cor['geometry']=uni_cor['to_zone_coordinates'].apply(loads)
uni_cor['geometry'].contains(

# Testing Script

In [201]:
print(ghk_order_withroute_gdf.loc[10,'Query XML']['AddressLookupResult']['SuggestedAddress']['Address']['PremisesAddress']['ChiPremisesAddress'])

{'Region': '新界', 'ChiDistrict': {'DcDistrict': '荃灣區'}, 'ChiEstate': {'EstateName': '象山邨'}, 'BuildingName': '樂山樓', 'Chi3dAddress': {'ChiFloor': {'FloorNum': '10'}, 'ChiUnit': {'UnitDescriptor': '室', 'UnitNo': '1034'}}}


In [ ]:
cnt=0
for i in range(len(red_cross_result_df)):
    parsed_dict=eval(route_df.loc[i,'Query XML'])
    dict_key=parsed_dict['AddressLookupResult']['SuggestedAddress']['Address']['PremisesAddress']['ChiPremisesAddress']
#     print(i,dict_key.keys())
    if 'BuildingName' in dict_key: #<--- Change this to show differet element
#         if isinstance(dict_key['BuildingName'],list):
#             print (i+1,dict_key['Chi3dAddress'][0])
        
#         else: 
            print(i+1,dict_key['BuildingName'])
    cnt+=1
print ("Total :",cnt)

In [167]:
for i,v in enumerate(address_ser):
    if re.search("[\u4e00-\u9fa5]",v):
        print(i,v)
    else: 
        print(i)
#check ['Address']['PremisesAddress']['EngPremisesAddress']['

0
1
2 新界 上水清曉路5號 清河邨清顯樓21樓 7室
3
4
5 新界 大埔泰亨邨泰亨1號 泰亨豪園  26座一樓1樓
6
7
8
9
10
11 粉嶺雞嶺村49號1F
12
13
14
15 天水圍天恆邨恆滿樓1154室
16 水邊圍邨湖水樓樓 537室
17 藍田徳田村德敬樓426室
18
19
20
21
22
23 新界 大埔大埔公路元洲仔段3821號 宏福苑  F座 宏昌閣2607
24
25
26
27
28
29
30 大埔太和邨居和樓3219室
31
32
33
34
35 新界 沙田 安景街23號 碧濤花園 第二期 B座 富臨閣 9樓 D室
36
37 新界 北區上水粉錦公路338號 歌賦嶺 龍柏徑16號屋


In [100]:
for i,v in enumerate(xml_ser):
    if i==2:
        if isinstance(v['AddressLookupResult']['SuggestedAddress'],list):
            for item,result in enumerate(v['AddressLookupResult']['SuggestedAddress']):
                print (i)
                print (f"{i}.{item}\n {result}")
                print ('-'*10)
            print ('Total:',item+1,"\n\n\n")
        else:
            print (i,'\n',v,"\n")

0
0.0
 {'Address': {'PremisesAddress': {'EngPremisesAddress': {'EngEstate': {'EstateName': 'THE HONG KONG JOCKEY CLUB HAPPY VALLEY CLUBHOUSE'}, 'EngStreet': {'StreetName': 'SHAN KWONG ROAD', 'BuildingNoFrom': '25'}, 'EngDistrict': {'DcDistrict': 'WAN CHAI DISTRICT'}, 'Region': 'HK'}, 'ChiPremisesAddress': {'Region': '香港', 'ChiDistrict': {'DcDistrict': '灣仔區'}, 'ChiStreet': {'StreetName': '山光道', 'BuildingNoFrom': '25'}, 'ChiEstate': {'EstateName': '香港賽馬會跑馬地會所'}}, 'GeoAddress': '3728914052T20050430', 'GeospatialInformation': {'Northing': '814071', 'Easting': '837286', 'Latitude': '22.26546', 'Longitude': '114.18675'}}}, 'ValidationInformation': {'Score': '71.43'}}
----------
0
0.1
 {'Address': {'PremisesAddress': {'EngPremisesAddress': {'BuildingName': 'THE HONG KONG JOCKEY CLUB HAPPY VALLEY CLUBHOUSE', 'EngStreet': {'StreetName': 'SHAN KWONG ROAD', 'BuildingNoFrom': '25'}, 'EngDistrict': {'DcDistrict': 'WAN CHAI DISTRICT'}, 'Region': 'HK'}, 'ChiPremisesAddress': {'Region': '香港', 'ChiDist

In [96]:
ghk_order_withroute_gdf.loc[14,'Query XML']['AddressLookupResult']['RequestAddress']['AddressLine']
ghk_order_df_copy=ghk_order_withroute_gdf.copy()
ghk_order_df_copy.head()

In [97]:
ghk_order_df_copy=ghk_order_df_copy.loc[448:450,:]
ghk_order_df_copy.apply(lambda x:address_query(x,"Address"),axis=1)
ghk_order_df_copy.reset_index(drop=True,inplace=True)

In [99]:
ghk_order_df_copy

,Delivery Date,Case No.,Patient Name,Patient Phone Number,Address,Formatted Address - Chinese,Region - Chinese,18-District - Chinese,Latitude,Longitude,Query XML,geometry,Price(Same Day),to_zone_name,Price(Speed),Price(Instant)


In [94]:
#Experiment
def address_query(df,column):
    param={"q":df[column],"Accept":"application/json","3d":"1","n":"5"} 
    # return the json of the top result with floor and unit detail
    q_url=requests.get('https://www.als.ogcio.gov.hk/lookup?',params=param)
    result=xmltodict.parse(q_url.text)
    df["Query XML"]=result
    return df

In [113]:
for index,result in enumerate(ghk_order_df['Query XML']):
    while index >0:
        if 'SuggestedAddress' in result['AddressLookupResult']:
            address_detail=result['AddressLookupResult']['SuggestedAddress']['Address']['PremisesAddress']
            score=result['AddressLookupResult']['SuggestedAddress']['ValidationInformation']['Score']
            #eng_address=address_detail['EngPremisesAddress']
            #chi_address=address_detail['ChiPremisesAddress']
            geo_address=address_detail['GeospatialInformation']
            #print (f"{index}\n English: \n{eng_address}\n Chinese: \n{chi_address}\n Lat,Long: \n{geo_address} \n Score: \n {score}")
            print (index,f"({geo_address['Latitude']},{geo_address['Longitude']})")
        else: 
            print(np.NAN)
        break

1 (22.24216,114.14897)
2 (22.30793,114.25649)
3 (22.42401,114.22675)
4 (22.46004,114.17255)
5 (22.4836,114.13887)
6 (22.29178,113.9492)
7 (22.32873,114.22903)
8 (22.34213,114.11023)
9 (22.44894,114.17555)
10 (22.31559,114.26296)
11 (22.30714,114.2504)
12 (22.32831,114.14795)
13 (22.50438,114.11176)
14 (22.31541,114.19015)
15 (22.44592,114.02744)
16 (22.29573,113.95347)
17 (22.29672,113.95402)
18 (22.42901,113.99914)
19 (22.244,114.15909)
20 (22.346,114.18652)
21 (22.36074,114.10307)
22 (22.45217,114.00534)
23 (22.33439,114.15655)
24 (22.35633,114.09524)
25 (22.36486,114.12709)
26 (22.49407,114.14094)
27 (22.2447,114.15124)
28 (22.34085,114.18877)
29 (22.29573,113.95347)
30 (22.34099,114.20253)
31 (22.34099,114.20253)


In [ ]:
for index,result in enumerate(ghk_order_df['Query XML']):
    while index <766:
        if 'SuggestedAddress' in result['AddressLookupResult']:
            address_detail=result['AddressLookupResult']['SuggestedAddress']['Address']['PremisesAddress']
            score=result['AddressLookupResult']['SuggestedAddress']['ValidationInformation']['Score']
            #eng_address=address_detail['EngPremisesAddress']
            #chi_address=address_detail['ChiPremisesAddress']
            geo_address=address_detail['GeospatialInformation']
            #print (f"{index}\n English: \n{eng_address}\n Chinese: \n{chi_address}\n Lat,Long: \n{geo_address} \n Score: \n {score}")
            print (index,f"({geo_address['Latitude']},{geo_address['Longitude']})")
        else: 
            print(np.NAN)
        break

In [114]:
for i,v in enumerate(ghk_order_df.loc[:,"Query XML"]):
    print (i,'\n',v)
    

0 
 {'AddressLookupResult': {'RequestAddress': {'AddressLine': 'Floor 32 Flat A BLK 1, HILLTOP GARDENS, 33 FUNG SHING STREET, KOWLOON'}, 'SuggestedAddress': {'Address': {'PremisesAddress': {'EngPremisesAddress': {'EngBlock': {'BlockDescriptor': 'BLK', 'BlockNo': '1', 'BlockDescriptorPrecedenceIndicator': 'Y'}, 'EngEstate': {'EstateName': 'HILLTOP GARDENS'}, 'EngStreet': {'StreetName': 'FUNG SHING STREET', 'BuildingNoFrom': '33'}, 'EngDistrict': {'DcDistrict': 'WONG TAI SIN DISTRICT'}, 'Region': 'KLN'}, 'ChiPremisesAddress': {'Region': '九龍', 'ChiDistrict': {'DcDistrict': '黃大仙區'}, 'ChiStreet': {'StreetName': '豐盛街', 'BuildingNoFrom': '33'}, 'ChiEstate': {'EstateName': '海港花園'}, 'ChiBlock': {'BlockDescriptor': '座', 'BlockNo': '1'}}, 'GeoAddress': '3982322578T20050430', 'GeospatialInformation': {'Northing': '822575', 'Easting': '839823', 'Latitude': '22.34226', 'Longitude': '114.21138'}}}, 'ValidationInformation': {'Score': '93.06'}}}}
1 
 {'AddressLookupResult': {'RequestAddress': {'Address